In [1]:
import os
os.chdir('./yolov5')

In [2]:
from yolov5.utils.dataloaders import LoadImages
from yolov5.utils.general import non_max_suppression, scale_boxes, xyxy2xywh
from yolov5.utils.augmentations import letterbox
from yolov5.models.experimental import attempt_load
import torch
import cv2
import numpy as np

class Yolov5_Predictor(object):
    def __init__(self, weight_path:str, data:str, img_size=(960, 960), device='cpu') -> None:
        self.device = torch.device(device) if isinstance(device, str) else device
        self.model = attempt_load(weights=weight_path, device=self.device, inplace=True, fuse=True)
        self.model = self.model.to(self.device)

        self.img_size = img_size
        self.stride = max(int(self.model.stride.max()), 32)
        self.classes_names = self.model.names
    def predict(self, img_path:str):
        im0 = cv2.imread(img_path)
        # Processing image
        im = letterbox(im0, self.img_size, stride=self.stride)[0]  # padded resize
        im = im.transpose((2, 0, 1))[::-1]  # HWC to CHW, BGR to RGB
        im = np.ascontiguousarray(im)
        im = torch.from_numpy(im).to(self.device)
        im = im.unsqueeze(dim=0) # add batch_size dimension
        im = im.float()  # uint8 to fp16/32
        im /= 255
        # Do Predict
        pred = self.model(im)

        conf_thres = 0.1
        iou_thres = 0.6
        max_det = 1000
        # Process predictions
        pred = non_max_suppression(pred, conf_thres, iou_thres, max_det=max_det)
        det = pred[0]
        if len(det):
            bboxs = []
            det[:, :4] = scale_boxes(im.shape[2:], det[:, :4], im0.shape).round()
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]
            for *xyxy, conf, cls in reversed(det):
                # xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                bboxs.append([int(cls), *xyxy]) # [cls, x1, y1, x2, y2]
            return bboxs
        else:
            return None

c:\Users\user\anaconda3\envs\yolov5\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

img_size = (960, 960)
weight_path = './runs/train/[2022-12-29-1549]960_bs16_med_E400/weights/best.pt'
data = './data/mydataset.yaml'
predictor = Yolov5_Predictor(weight_path, data, img_size)

Fusing layers... 
Model summary: 212 layers, 20861016 parameters, 0 gradients, 47.9 GFLOPs


In [12]:
from os.path import join
from tqdm import tqdm
start_idx = 5
end_idx = start_idx + 3

category = predictor.classes_names[2]

if category == 'scratch':
    expand = 100
else:
    expand = 80

split = 'Train'
output_root = '../data/stage2/{}/images/'.format(split)
os.makedirs(output_root, exist_ok=True)

unet_img_root = '../data/unet/{}/images/'.format(split)
os.makedirs(unet_img_root, exist_ok=True)
unet_mask_root = '../data/unet/{}/masks/'.format(split)
os.makedirs(unet_mask_root, exist_ok=True)
# for i in range(start_idx, end_idx):
for i in tqdm(range(100)):
    file_name = '{}_{}.png'.format(category, i)
    img_path = '../data/yolov5/{}/images/'.format(split) + file_name
    result = predictor.predict(img_path)

    mask_path = '../data/yolov5/{}/masks/'.format(split) + file_name
    im = cv2.imread(img_path)
    mask = cv2.imread(mask_path)
    im_h, im_w, c = im.shape
    new_mask = np.zeros((im_h, im_w, c), dtype=np.uint8)
    ratios = []
    total_area = im_h * im_w
    for box in result:
        cls_idx, x1, y1, x2, y2 = box
        ratio = (x2 - x1) / (y2 - y1)
        if ratio < 1:
            ratio = 1 / ratio
        ratios.append(ratio)

        area = ((x2 - x1) * (y2 - y1)) / total_area
        if area <= 0.015:
            cur_expand = expand * 2
        else:
            cur_expand = expand
        x1 = max(int(x1) - cur_expand, 0)
        x2 = min(int(x2) + cur_expand, im_w)
        y1 = max(int(y1) - cur_expand, 0)
        y2 = min(int(y2) + cur_expand, im_h)

        # crop = im[int(y1):int(y2), int(x1):int(x2), :]
        cv2.rectangle(im, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 4, cv2.LINE_AA)
        cv2.rectangle(mask, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 4, cv2.LINE_AA)
        new_mask[y1 : y2, x1 : x2, ...] = 255

    # print(np.mean(ratios))
    new_size = (im_h // 6, im_w // 6)
    im = cv2.resize(im, new_size)
    mask = cv2.resize(mask, new_size)
    # new_mask = cv2.resize(new_mask, new_size)
    # cv2.imshow('image', cv2.resize(new_mask, new_size))
    # cv2.waitKey(0)
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(new_mask[...,0], connectivity=8, ltype=cv2.CV_16U)
    # print(stats)

    if num_labels >= 1:
        stats[:,2] += stats[:,0]
        stats[:,3] += stats[:,1]
        new_im = cv2.imread(img_path)
        new_mask = cv2.imread(mask_path, 0)
        unique = np.unique(new_mask)
        ret, new_mask = cv2.threshold(new_mask, unique[1] - 1, 255, cv2.THRESH_BINARY)

        raw_name = file_name.split('.')[0]
        for i in range(1, num_labels):
            new_name = raw_name + '_{}.png'.format(i - 1)
            x1, y1, x2, y2, area = stats[i]
            x1 = max(int(x1), 0)
            x2 = min(int(x2), im_w)
            y1 = max(int(y1), 0)
            y2 = min(int(y2), im_h)
            cv2.imwrite(join(unet_img_root, new_name), new_im[y1:y2, x1:x2, :])
            cv2.imwrite(join(unet_mask_root, new_name), new_mask[y1:y2, x1:x2])

    # if num_labels >= 1:
    #     stats[:,2] += stats[:,0]
    #     stats[:,3] += stats[:,1]
        # new_im = cv2.imread(img_path)
        # new_mask = cv2.imread(mask_path)
        new_mask = cv2.imread(mask_path)
        for i in range(1, num_labels):
            x1, y1, x2, y2, area = stats[i]
            x1 = max(int(x1), 0)
            x2 = min(int(x2), im_w)
            y1 = max(int(y1), 0)
            y2 = min(int(y2), im_h)
            cv2.rectangle(new_im, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 4, cv2.LINE_AA)
            cv2.rectangle(new_mask, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 4, cv2.LINE_AA)

        new_im = cv2.resize(new_im, new_size)

        new_mask = cv2.resize(new_mask, new_size)

        cv2.imwrite(os.path.join(output_root, file_name), cv2.vconcat([cv2.hconcat([mask, im]), cv2.hconcat([new_im, new_mask])]))
    # cv2.imshow('image', cv2.vconcat([cv2.hconcat([mask, im]), cv2.hconcat([new_im, new_mask])]))
    # cv2.waitKey(0)

100%|██████████| 100/100 [01:15<00:00,  1.32it/s]


array([[       0,        0,     3384,     3330, 10884834],
       [     959,      842,      202,      434,    87668],
       [     983,     1404,      193,      265,    51145],
       [     871,     1866,      611,      413,   245073]], dtype=int32)

In [ ]:
'../data/yolov5/Val/masks/powder_uncover_1.png'